<a href="https://colab.research.google.com/github/cff100/MecFLu2/blob/main/C%C3%B3digo_Atualizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdsapi
!pip install cartopy
!pip install netcdf4
!pip install openpyxl


  Preparing metadata (setup.py) ... done
  Created wheel for multiurl: filename=multiurl-0.3.1-py3-none-any.whl size=21131 sha256=004d0d1a7403a6f9248af65e70e479dd84308c14ef14e28eb23e2ac10e1e822e
  Stored in directory: /root/.cache/pip/wheels/9b/d9/5f/36a39fd10d15b5b2d362ad6dc8a1bd28a3b1e14e08357944bf
Successfully built multiurl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.3 MB/s eta 0:00:00


In [2]:
import math
import cdsapi
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
!pip install --upgrade cdsapi

In [3]:
# Constantes
PA = 101.325  # Pressão atmosférica ao nível do mar em kPa
k = 1.15e-4   # Constante em s²/m²
g = 9.81      # Aceleração gravitacional em m/s²

# Função para calcular a altura a partir da pressão
def calcular_altura(PA, k, P_h, g):
    # P_h: Pressão a uma altura h (em KPa)
    altura = -math.log(P_h / PA) / (k * g)
    return altura

# Função para converter horário de Brasília para UTC
def brasilia_to_utc(hour_brasilia):
    brasilia_time = datetime.strptime(hour_brasilia, '%H:%M')
    utc_time = brasilia_time + timedelta(hours=3)
    return utc_time.strftime('%H:%M')

# Função para converter temperatura de Kelvin para Celsius
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15


# Níveis de pressão em KPa
pressure_levels = [70.0, 75.0, 77.5, 80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5, 100.0]

# Coordenadas das plataformas
platforms = {
    'NAMORADO 2 (PNA-2)': (-22.45073, -40.41175),
    'PETROBRAS 26 (P-26)': (-22.4684, -40.02869),
    'PETROBRAS 32 (P-32)': (-22.2051, -40.1431),
    'PETROBRAS 37 (P-37)': (-22.4868, -40.09779),
    'PETROBRAS IX': (-22.57358, -40.82192),
    'PETROBRAS XIX': (-22.3927, -40.05438),
    'PETROBRAS XXXIII': (-22.37, -40.0267),
    'VERMELHO 1 (PVM-1)': (-22.16065, -40.27872),
    'VERMELHO 2 (PVM-2)': (-22.17535, -40.29147),
}


In [4]:
# Hora de Brasília a ser convertida
brasilia_hour = '03:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)


2024-09-04 10:24:17,542 INFO Request ID is e7002cfd-d8bf-4d47-a28b-70ca16306d4f
INFO:cads_api_client.processing:Request ID is e7002cfd-d8bf-4d47-a28b-70ca16306d4f
2024-09-04 10:24:17,716 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:25:07,122 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


c4506f12b8ba999dcae14c5a7d53cef3.nc:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

2024-09-04 10:25:11,138 INFO Request ID is 6faa30ff-59fa-4f3c-b1f8-3d96f337fd94
INFO:cads_api_client.processing:Request ID is 6faa30ff-59fa-4f3c-b1f8-3d96f337fd94
2024-09-04 10:25:11,312 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:25:43,455 INFO status has been updated to running
INFO:cads_api_client.processing:status has been updated to running
2024-09-04 10:26:01,032 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


f52d89231551cee43d7532b87bad85bd.nc:   0%|          | 0.00/3.05M [00:00<?, ?B/s]

2024-09-04 10:26:04,525 INFO Request ID is ddae7018-8e41-411f-9854-00078d18083b
INFO:cads_api_client.processing:Request ID is ddae7018-8e41-411f-9854-00078d18083b
2024-09-04 10:26:04,683 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:26:36,849 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

ab0e23201d4dcf3f93c0993a7ae913e8.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 10:33:41,635 INFO Request ID is ffcfa9b6-7c0d-4fc0-a0f7-ec9e89a8a010
INFO:cads_api_client.processing:Request ID is ffcfa9b6-7c0d-4fc0-a0f7-ec9e89a8a010
2024-09-04 10:33:41,794 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:34:57,034 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


120e0a098aee984964617f56d721bf78.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 10:35:00,604 INFO Request ID is 9483cbd2-4c88-4f02-8aa7-e10b3e1b22ad
INFO:cads_api_client.processing:Request ID is 9483cbd2-4c88-4f02-8aa7-e10b3e1b22ad
2024-09-04 10:35:00,759 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:35:50,176 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


524314e96ebb7611ec95d3c104ba65d9.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

In [5]:
# Hora de Brasília a ser convertida
brasilia_hour = '09:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)

2024-09-04 10:35:53,905 INFO Request ID is 6613cc55-617c-4c87-ac64-a4480b0a0702
INFO:cads_api_client.processing:Request ID is 6613cc55-617c-4c87-ac64-a4480b0a0702
2024-09-04 10:35:54,068 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:37:09,287 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


cd5ab8f91bd51b5314fdadae5528f62f.nc:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

2024-09-04 10:37:12,502 INFO Request ID is 08eef106-ddc8-4d05-9303-89f67aab629e
INFO:cads_api_client.processing:Request ID is 08eef106-ddc8-4d05-9303-89f67aab629e
2024-09-04 10:37:12,659 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:38:27,861 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


8c6e356444940bffbaf4bcc3d67eba9c.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

2024-09-04 10:38:31,544 INFO Request ID is 8dc559e0-3c2f-45d5-8f68-c2d1d1475f12
INFO:cads_api_client.processing:Request ID is 8dc559e0-3c2f-45d5-8f68-c2d1d1475f12
2024-09-04 10:38:31,713 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:39:21,172 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

4427dfb1d25d152e5f681d8a49734054.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 10:46:07,191 INFO Request ID is 07900915-ff73-4e3f-82ef-be04c60d46f7
INFO:cads_api_client.processing:Request ID is 07900915-ff73-4e3f-82ef-be04c60d46f7
2024-09-04 10:46:07,344 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:46:27,938 INFO status has been updated to running
INFO:cads_api_client.processing:status has been updated to running
2024-09-04 10:46:39,579 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


bbd44206b5e0225990fda20d47e74a29.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 10:46:42,564 INFO Request ID is a9933131-3f15-4905-b797-68401b39fbae
INFO:cads_api_client.processing:Request ID is a9933131-3f15-4905-b797-68401b39fbae
2024-09-04 10:46:42,721 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:47:15,020 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


8c22bb400a61d52427493446a8da43e.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

In [6]:
# Hora de Brasília a ser convertida
brasilia_hour = '15:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)


2024-09-04 10:47:18,639 INFO Request ID is 58159ed1-9016-4d10-936f-bdc8e89fe5e6
INFO:cads_api_client.processing:Request ID is 58159ed1-9016-4d10-936f-bdc8e89fe5e6
2024-09-04 10:47:18,801 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:47:39,409 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


b12355714885c5979b7edbf597384d19.nc:   0%|          | 0.00/3.01M [00:00<?, ?B/s]

2024-09-04 10:47:42,933 INFO Request ID is f3e76868-046a-431a-ba73-deb2faf40969
INFO:cads_api_client.processing:Request ID is f3e76868-046a-431a-ba73-deb2faf40969
2024-09-04 10:47:43,093 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:48:03,700 INFO status has been updated to running
INFO:cads_api_client.processing:status has been updated to running
2024-09-04 10:48:15,263 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


78f5be7656a22eba829172add3955723.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 10:48:19,095 INFO Request ID is 2c89f7c6-dbdc-4f00-8570-b145abb654f8
INFO:cads_api_client.processing:Request ID is 2c89f7c6-dbdc-4f00-8570-b145abb654f8
2024-09-04 10:48:19,273 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:48:51,419 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

b2ac1755aeee6ba242abf0ce8e44394c.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 10:52:34,137 INFO Request ID is b8c3b034-7f6b-4b39-bfcd-b86dd26b7aa5
INFO:cads_api_client.processing:Request ID is b8c3b034-7f6b-4b39-bfcd-b86dd26b7aa5
2024-09-04 10:52:34,300 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:53:06,439 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


f3480c692dec92737444ec103a206e6a.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

2024-09-04 10:53:09,725 INFO Request ID is af82f44c-c703-4a36-a5ec-cd8a6d3a3bfa
INFO:cads_api_client.processing:Request ID is af82f44c-c703-4a36-a5ec-cd8a6d3a3bfa
2024-09-04 10:53:09,883 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:53:42,057 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


fc6e3de3730c6c1c6b8990b2b20dcb91.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

In [7]:
# Hora de Brasília a ser convertida
brasilia_hour = '21:00'
utc_hour = brasilia_to_utc(brasilia_hour)

# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()


# Inicializa a API do CDS
c = cdsapi.Client(url='https://cds-beta.climate.copernicus.eu/api', key='89647e12-cf97-495d-a31b-3c325c5fd352')

# Loop sobre cada nível de pressão
for level in pressure_levels:
    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = f'Era5_2023_{int(level*10)}HPA_CAMPOS_{brasilia_hour}.nc'

    # Requisição dos dados
    dataset = 'reanalysis-era5-pressure-levels'
    request = {
    'product_type': ['reanalysis'],
    'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind'],
    'year': ['2023'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
    'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'],
    'time': [utc_hour],
    'area': [-27, -45, -20, -35],  # Coordenadas da Bacia de Campos
    'pressure_level': f'{int(level*10)}',  # Converte o nível de kPa para hPa
    'data_format': 'netcdf',
    'download_format': 'unarchived'
}

    c.retrieve(dataset, request, nc_filename)

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='netcdf4')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Calcula a altura correspondente ao nível de pressão atual
    altura = calcular_altura(PA, k, level, g)

    # Cria um DataFrame para armazenar os resultados específicos para o nível de pressão atual
    result_df = pd.DataFrame(index=platforms.keys())

    season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
    }

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:
        for platform, (lat, lon) in platforms.items():

            # Interpola o valor da média da velocidade do vento
            wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Armazena os valores de vento e temperatura no DataFrame
            result_df.loc[platform, f'Velocidade_Vento_{season_names[season]}'] = wind_value
            result_df.loc[platform, f'Temperatura_K_{season_names[season]}'] = temp_kelvin
            result_df.loc[platform, f'Temperatura_C_{season_names[season]}'] = temp_celsius



    # Adiciona as colunas de pressão e altura
    result_df['Nível_de_Pressão'] = f'{int(level*10)} HPA'
    result_df['Altitude_(m)'] = altura
    result_df['Horário_Brasília'] = brasilia_hour
    result_df['Horário_UTC'] = utc_hour

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    combined_df = pd.concat([combined_df, result_df])

    # Fecha o arquivo NetCDF
    ds.close()

# Ajusta a ordem das colunas
combined_df = combined_df[['Nível_de_Pressão', 'Altitude_(m)', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_Verão', 'Temperatura_K_Verão', 'Temperatura_C_Verão',
                           'Velocidade_Vento_Outono', 'Temperatura_K_Outono', 'Temperatura_C_Outono',
                           'Velocidade_Vento_Inverno', 'Temperatura_K_Inverno', 'Temperatura_C_Inverno',
                           'Velocidade_Vento_Primavera', 'Temperatura_K_Primavera', 'Temperatura_C_Primavera']]


# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_Combined_Results_{brasilia_hour}.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_Combined_Results_{brasilia_hour}.xlsx'
combined_df.to_excel(excel_filename, index=True)


2024-09-04 10:53:45,776 INFO Request ID is 68ab1f48-7a4c-4ce9-9880-cad56898a6bc
INFO:cads_api_client.processing:Request ID is 68ab1f48-7a4c-4ce9-9880-cad56898a6bc
2024-09-04 10:53:45,933 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:54:18,095 INFO status has been updated to running
INFO:cads_api_client.processing:status has been updated to running
2024-09-04 10:54:35,354 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


61acc64dc8f64455e145365364da8d9c.nc:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

2024-09-04 10:54:38,404 INFO Request ID is cc37b55f-12c4-4d91-88be-3f73e3031406
INFO:cads_api_client.processing:Request ID is cc37b55f-12c4-4d91-88be-3f73e3031406
2024-09-04 10:54:38,566 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:55:10,721 INFO status has been updated to running
INFO:cads_api_client.processing:status has been updated to running
2024-09-04 10:55:27,981 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


37c3b85867300139c615b68162addff0.nc:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

2024-09-04 10:55:31,134 INFO Request ID is de2b4384-0461-42e2-869c-6d3d84fad70e
INFO:cads_api_client.processing:Request ID is de2b4384-0461-42e2-869c-6d3d84fad70e
2024-09-04 10:55:31,299 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 10:56:20,713 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


...

86189f4358d76dfa2c15c281b3bf418.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 11:07:07,217 INFO Request ID is c8823e9e-e768-46d2-9336-f792f0b0fa95
INFO:cads_api_client.processing:Request ID is c8823e9e-e768-46d2-9336-f792f0b0fa95
2024-09-04 11:07:07,381 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 11:08:22,620 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


9b4b4754db2759749fc5a3c3569ab9d9.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

2024-09-04 11:08:25,914 INFO Request ID is 62d363ad-3754-4cf5-8142-7a6b83f0d93a
INFO:cads_api_client.processing:Request ID is 62d363ad-3754-4cf5-8142-7a6b83f0d93a
2024-09-04 11:08:26,078 INFO status has been updated to accepted
INFO:cads_api_client.processing:status has been updated to accepted
2024-09-04 11:09:15,514 INFO status has been updated to successful
INFO:cads_api_client.processing:status has been updated to successful


cd206b52cfb8723b541e51bb1b7fe4d1.nc:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

In [8]:
import pandas as pd
import glob

# Lista de horários na ordem desejada
horarios_brasilia = ['03:00', '09:00', '15:00', '21:00']

# Lista para armazenar os DataFrames na ordem correta
dataframes = []

# Loop sobre cada horário e carregar o DataFrame correspondente
for horario in horarios_brasilia:
    # Buscar o arquivo CSV correspondente ao horário atual
    csv_file = glob.glob(f'2023_Combined_Results_{horario}.csv')

    if csv_file:
        # Carregar o DataFrame e adicionar à lista
        df = pd.read_csv(csv_file[0])
        dataframes.append(df)

# Concatenar os DataFrames na ordem correta
combined_df = pd.concat(dataframes, ignore_index=True)


# Salvar o DataFrame combinado como CSV
combined_df.to_csv('2023_Combined_Results.csv', index=False)

# Salvar o DataFrame combinado como Excel
combined_df.to_excel('2023_Combined_Results.xlsx', index=False)
